In [1]:
import pandas as pd #Se importa la libreria pandas 
df=pd.read_csv('calles_de_medellin_con_acoso.csv', delimiter = ';') #Se lee el data frame 
df['name'] = df['name'].str.lower()   #Se pone en minúsucula todas las filas de la columna 'name'
df["harassmentRisk"]=df["harassmentRisk"].interpolate()  #Interpolar en la columna harassmentRisk para llenar los datos que sean NaN
df['peso'] = (df["length"]**(2*df["harassmentRisk"]))   #Hago una nueva columna que combine el length y harrassmentRisk (variable v), que luego será el peso de cada calle
df=df.dropna() #Elimino las filas que tengan algún dato nulo, en este caso en la columna 'name'

In [2]:
origenes=list(df["origin"]) #Lista con todos los origenes
destinos=list(df["destination"]) #Lista con todos los destinos

In [3]:
for element in destinos: #Para cada elemento en destinos
    if element not in origenes: #Si el elemento no está en origenes
        origenes.append(element) #Lo agrego

In [4]:
conexiones=[] # Lista que contendra las conexiones de cada uno de los elementos en origenes
pesos=[] #Lista que contendra el peso de las conexiones de cada uno de los elementos en origenes
for origen in origenes: #Para cada origen en origenes
    c=list(df.query("origin==@origen")["destination"]) #Filtro el dataset y hago una lista de los destinos que tienen como origin el origen
    p=list(df.query("origin==@origen")["peso"]) #Filtro el dataset y hago una lista de los pesos que tienen como origin el origen
    conexiones.append(c) #Agrego las conexiones a la lista
    pesos.append(p) #Agrego los pesos a la lista

In [28]:
'''
Cada elemnto en conexiones y en pesos es una lista que contiene estos datos para cada elemento de origenes
Al comparar sus longitudes, verificamos que a cada elemento en origenes le corresponda un elemento en conexiones y uno en pesos
'''
len(origenes)==len(conexiones)==len(pesos) 

True

In [29]:
diccionario={} #Creo la lista de adyacencia
for i in range(len(origenes)): #Para cada posición en origenes,conexiones y pesos
    diccionario[origenes[i]]=[conexiones[i],pesos[i]] #hago la posición de origenes la llave, la de conexiones la primera lista de los valores y la de pesos la segunda

In [30]:
'''Esta clase define los vértices del grafo'''
class Vertice:

    def __init__ (self, v):   #Defino el constructor de la clase vértice
        self.id = v   #Instancio la etiqueta del vértice
        self.adyacentes = []   #Instancio la lista de los vertices adyacentes a v
        self.visitado = False   #Marco el vértice como no visitado
        self.predecesor = None   #Defino el vértice sin predecesor
        self.peso = float('inf')   #Defino el peso para llegar al vértice en infinito

    def add_adyacente(self, v, p ):   # Función para agregar los vertices que son adyacentes a v con sus respectivos pesos p
        if v not in self.adyacentes:
            self.adyacentes.append([v,p])

In [31]:
'''Clase que define el grafo'''
class Graph:
    
    def __init__(self):   #Defino el constructor de la clase Graph
        self.vertices = {}   #Instancio el diccionario que usará la clase

    def add_vertice(self, id):  # Función para agregar el vértice al {}
        if id not in self.vertices:  #Verifico que el id no esté en {} para poder agregarlo y que no hayan repetidos
            self.vertices[id] = Vertice(id)

    def add_arista(self, v1, v2, p):   #Función para agregar una arista entre dos nodos v1,v2 y su peso p
        if v1 in self.vertices and v2 in self.vertices:   #verifico que los dos vertices existan
            self.vertices[v1].add_adyacente(v2,p)   #añado la arista entre v1 y v2 con peso p
            #self.vertices[v2].add_adyacente(v1,p)  
    
    def imprimir(self): #Función para imprimir los valores finales de la gráfica
        for v in self.vertices:
            print("El peso para llegar al vértice " , v , " es " , self.vertices[v].peso , " llegando desde " , self.vertices[v].predecesor)

    def camino(self, v1, v2): #Función para imprimir el camino a seguir y su costo
        camino = []
        actual = v2
        while actual != None:
            camino.insert(0, actual)
            actual = self.vertices[actual].predecesor
        return [camino, self.vertices[v2].peso]

    def minimo(self, lista): # Función para elegir el siguiente vértice que tena el menor peso para usar en el algoritmo Dijkstra
        if len(lista) > 0:
            m = self.vertices[lista[0]].peso
            v = lista[0]
            for e in lista:
                if m > self.vertices[e].peso:
                    m = self.vertices[e].peso
                    v = e
            return v
    
    def dijkstra(self, inicio):  #Algoritmo para determinar el camino más corto
        if inicio in self.vertices: #Verifico que el vértice en que voy a iniciar exista
            self.vertices[inicio].peso = 0  #Defino su peso en 0
            actual = inicio  #Lo hago el vértice actual
            NoVisitados = []   #Instancio la lista de nodos Novisitados vacía

            for v in self.vertices: #Con este for recorro cada vértice del grafo
                if v != inicio: # Tomo todos los vértices diferentes del que va a ser el inicio
                    self.vertices[v].peso = float('inf')   # Defino su peso como infinito
                self.vertices[v].predecesor = None   #Defino su predecesor como None
                NoVisitados.append(v)   #Agrego el vértice a la lista NoVisitados

            while len(NoVisitados) > 0:  #Mientras que la lista de nodos no esté vacía 
                for vecino in self.vertices[actual].adyacentes:  #Para cada vértice adyacente al actual 
                    if self.vertices[vecino[0]].visitado == False: # Si el adyacente no ha sido visitado
                        if self.vertices[actual].peso + vecino[1] < self.vertices[vecino[0]].peso: #Verifico si el peso del actual + el de la arista es menor al peso del vértice
                            self.vertices[vecino[0]].peso = self.vertices[actual].peso + vecino[1] #Actualizo el nuevo peso del vértice adyacente
                            self.vertices[vecino[0]].predecesor = actual   #Hago el predecesor del vértice adyacente = al vértice actual

                self.vertices[actual].visitado = True #Una vez tomados todos los nodos adyacentes, marco el nodo actual como visitado
                NoVisitados.remove(actual)   #Lo elimino de la lista de NoVisitados

                actual= self.minimo(NoVisitados)  #Hago que el vértice actual sea el que esté en la lista NoVisitados y tenga el menor peso 
                
        else:
            return False   #Return si el nodo de inicio no existe


In [32]:
g=Graph() # Instancio a g como un objeto de la clase Graph

for llave in origenes:   #Para cada llave
    g.add_vertice(llave)   #La agrego como un nuevo vértice

for llave in origenes:   #Para cada llave
    if len(diccionario[llave]) != 0:   #Si la llave tiene un valor(está conectada con más vértices)
        for i in range(len(diccionario[llave][0])):
            for valor in diccionario[llave][0]:   #Para cada vértice con el que está conectado
                g.add_arista(llave,diccionario[llave][0][i],diccionario[llave][1][i])   #Agrego una arista entre la llave y el vértice con su respectivo peso
                

In [33]:
print("\n\nLa ruta más rápida por Dijkstra junto a su costo es: ")
g.dijkstra("(-75.587803, 6.2103263)")
print(g.camino("(-75.587803, 6.2103263)","(-75.5879378, 6.2104136)"))
print("\nLos valores finales de la gráfica son los siguientes: ")
g.imprimir()



La ruta más rápida por Dijkstra junto a su costo es: 
[['(-75.587803, 6.2103263)', '(-75.5879378, 6.2104136)'], 171.046842523728]

Los valores finales de la gráfica son los siguientes: 
El peso para llegar al vértice  (-75.5728593, 6.2115169)  es  38479.37031547771  llegando desde  (-75.5731414, 6.2107429)
El peso para llegar al vértice  (-75.5705202, 6.2106275)  es  38307.36795007301  llegando desde  (-75.5706449, 6.2106721)
El peso para llegar al vértice  (-75.5687719, 6.2099661)  es  38068.34381522007  llegando desde  (-75.5688512, 6.2099957)
El peso para llegar al vértice  (-75.5674348, 6.2094357)  es  38060.760655446444  llegando desde  (-75.5675817, 6.2090206)
El peso para llegar al vértice  (-75.5666527, 6.2091202)  es  38077.77415918471  llegando desde  (-75.5666978, 6.2090146)
El peso para llegar al vértice  (-75.5660756, 6.2089072)  es  38096.74471356531  llegando desde  (-75.5662079, 6.2089626)
El peso para llegar al vértice  (-75.5686884, 6.2063927)  es  37977.38787295713